In [1]:
import os
import shutil
import random
from tqdm import tqdm

def create_train_val_subset(source_dir, dest_dir, total_samples, split_ratio=0.8, seed=42):
    """
    Tạo một tập dữ liệu con có cấu trúc train/val từ một nguồn dữ liệu gốc.

    Args:
        source_dir (str): Đường dẫn đến thư mục dữ liệu gốc (ví dụ: '.../UDTIRI-Crack Detection/train').
        dest_dir (str): Đường dẫn đến thư mục đích, nơi tập dữ liệu con sẽ được tạo.
        total_samples (int): Tổng số lượng mẫu (cặp ảnh/mask) cần lấy.
        split_ratio (float): Tỷ lệ dữ liệu cho tập train (ví dụ: 0.8 cho 80% train, 20% val).
        seed (int): Seed cho việc chọn ngẫu nhiên để có thể lặp lại kết quả.
    """
    random.seed(seed)
    print(f"--- Bắt đầu tạo tập dữ liệu con tại: '{dest_dir}' ---")

    source_img_dir = os.path.join(source_dir, 'image')
    source_mask_dir = os.path.join(source_dir, 'label')

    # 1. Tạo cấu trúc thư mục đích
    dest_train_img_dir = os.path.join(dest_dir, 'train', 'image')
    dest_train_mask_dir = os.path.join(dest_dir, 'train', 'label')
    dest_val_img_dir = os.path.join(dest_dir, 'val', 'image')
    dest_val_mask_dir = os.path.join(dest_dir, 'val', 'label')

    for path in [dest_train_img_dir, dest_train_mask_dir, dest_val_img_dir, dest_val_mask_dir]:
        os.makedirs(path, exist_ok=True)
    print("Đã tạo cấu trúc thư mục train/val.")

    # 2. Lấy danh sách tất cả các file từ nguồn
    all_img_files = sorted(os.listdir(source_img_dir))
    all_mask_files = sorted(os.listdir(source_mask_dir))
    
    # Đảm bảo các file được ghép cặp đúng
    assert len(all_img_files) == len(all_mask_files), "Số lượng ảnh và mask trong thư mục nguồn không khớp!"
    
    # Tạo danh sách các chỉ số từ 0 đến N-1
    indices = list(range(len(all_img_files)))
    random.shuffle(indices) # Xáo trộn các chỉ số một cách ngẫu nhiên

    # 3. Chọn ra một tập con và chia train/val
    num_total_available = len(all_img_files)
    if total_samples > num_total_available:
        print(f"Cảnh báo: Bạn yêu cầu {total_samples} mẫu, nhưng chỉ có {num_total_available} mẫu. Sẽ sử dụng tất cả.")
        total_samples = num_total_available
        
    selected_indices = indices[:total_samples]
    
    num_train = int(total_samples * split_ratio)
    train_indices = selected_indices[:num_train]
    val_indices = selected_indices[num_train:]

    print(f"Tổng cộng lấy {total_samples} mẫu.")
    print(f"  - {len(train_indices)} mẫu cho tập train ({split_ratio*100:.0f}%)")
    print(f"  - {len(val_indices)} mẫu cho tập val ({(1-split_ratio)*100:.0f}%)")

    # 4. Hàm trợ giúp để sao chép file
    def copy_files(file_indices, dest_img_path, dest_mask_path, desc=""):
        for i in tqdm(file_indices, desc=f"Copying {desc} files"):
            img_filename = all_img_files[i]
            mask_filename = all_mask_files[i]
            
            # Sao chép ảnh
            shutil.copy2(os.path.join(source_img_dir, img_filename), os.path.join(dest_img_path, img_filename))
            # Sao chép mask
            shutil.copy2(os.path.join(source_mask_dir, mask_filename), os.path.join(dest_mask_path, mask_filename))

    # 5. Thực hiện sao chép
    copy_files(train_indices, dest_train_img_dir, dest_train_mask_dir, "train")
    copy_files(val_indices, dest_val_img_dir, dest_val_mask_dir, "val")

    print("\n✅ Hoàn thành!")
    print(f"Đã tạo thành công tập dữ liệu con tại '{dest_dir}'.")


if __name__ == '__main__':
    # --- CẤU HÌNH ---
    # Đường dẫn đến dữ liệu GỐC (chỉ cần thư mục cha chứa 'image' và 'label')
    # Chúng ta sẽ lấy dữ liệu từ tập train gốc để tạo ra tập con.
    SOURCE_DATA_DIR = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train'

    # Đường dẫn bạn muốn LƯU thư mục con mới
    # Thư mục này sẽ được tạo tự động với cấu trúc train/val bên trong.
    SUBSET_DESTINATION_DIR = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\my_crack_subset'
    
    # Tổng số ảnh bạn muốn có trong tập con
    TOTAL_SAMPLES_TO_CREATE = 50

    # Tỷ lệ chia: 0.8 nghĩa là 80% cho train, 20% cho validation
    TRAIN_VAL_SPLIT_RATIO = 0.8
    
    # --- CHẠY HÀM ---
    create_train_val_subset(
        source_dir=SOURCE_DATA_DIR,
        dest_dir=SUBSET_DESTINATION_DIR,
        total_samples=TOTAL_SAMPLES_TO_CREATE,
        split_ratio=TRAIN_VAL_SPLIT_RATIO,
        seed=42 # Giữ seed để có thể tạo lại chính xác tập dữ liệu này
    )

--- Bắt đầu tạo tập dữ liệu con tại: 'C:\Users\Admin\Documents\Python Project\DPL Crack detection\my_crack_subset' ---
Đã tạo cấu trúc thư mục train/val.
Tổng cộng lấy 50 mẫu.
  - 40 mẫu cho tập train (80%)
  - 10 mẫu cho tập val (20%)


Copying val files: 100%|██████████| 10/10 [00:00<00:00, 431.65it/s]


✅ Hoàn thành!
Đã tạo thành công tập dữ liệu con tại 'C:\Users\Admin\Documents\Python Project\DPL Crack detection\my_crack_subset'.
